In [ ]:
# %pip install crewai==0.55.2 crewai-tools==0.12.1
# %pip install google-ai-generativelanguage==0.6.9

# %pip install protobuf==5.26.1
# %pip install langchain-core==0.2.43
!pip list | grep lang

In [1]:
import os
import sys
import datetime
import pprint

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/crewAI/..


In [2]:
from models.llm import LLM

model = LLM('gpt-4o-mini')
llm = model.load_model()
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10ae1f130>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10ae50a90>, root_client=<openai.lib.azure.AzureOpenAI object at 0x107dd5180>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x10ae1f160>, temperature=0.0, max_tokens=4096, deployment_name='gpt-4o-mini', openai_api_version='2023-05-15')

In [3]:
llm.invoke("What is the capital of India?")

AIMessage(content='The capital of India is New Delhi.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5154047bf2', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-5dfd16f4-8dc2-4e5d-b68c-14788541e469-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22})

In [4]:
os.environ["OTEL_SDK_DISABLED"] = "true"
os.environ["CREWAI_TELEMETRY_OPT_OUT"] = "true"

### Search Tools

#### Tavily Search

1000 search credit / month

In [5]:
from langchain_community.tools import TavilySearchResults

tavily_search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=False,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)


In [ ]:
results = tavily_search.invoke({"query": "What happened with the SENSEX yesterday?"})
pprint.pp(results)

#### Serper Search

Total: 2500 search credit free

In [11]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper_search = GoogleSerperAPIWrapper()

In [ ]:
results = serper_search.results("Studies on cardiovascular impacts of weight loss drug from Lilly?")


### Crew with single agent and single task

In [6]:
from crewai import Agent, Task, Crew

search_tool = tavily_search

research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  llm=llm,
  tools=[search_tool],
  verbose=True
)

task = Task(
  description='Find and summarize the latest AI news',
  expected_output='A bullet list summary of the top 5 most important AI news',
  agent=research_agent,
)

crew = Crew(
    agents=[research_agent],
    tasks=[task],
    verbose=True,
    iterations=3
)

result = crew.kickoff()
print(result)

2025-01-03 09:24:42,225 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.
2025-01-03 09:24:42,229 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.
2025-01-03 09:24:42,232 - 8638448192 - manager.py-manager:293 - WARNING: Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 [2025-01-03 09:24:42][DEBUG]: == Working Agent: Researcher
 [2025-01-03 09:24:42][INFO]: == Starting Task: Find and summarize the latest AI news
I need to gather the latest AI news to provide a comprehensive summary. I will use the search tool to find the most recent and relevant information.

Action: tavily_search_results_json  
Action Input: {"query":"latest AI news"}  


2025-01-03 09:24:51,353 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.


 

[{'url': 'https://www.nbcnews.com/artificial-intelligence', 'content': "Artificial intelligence news: Chat AI, ChatGPT, AI generator, AI Chatbot and Bard | NBC News NBC News Now The latest news and top stories on artificial intelligence, including ChatGPT, AI Chatbot and Bard. AI NEWS Justine Goode / NBC News / Getty Images NBC News NOW NBC News NOW NBC News NOW NBC News NOW NBC News NOW NBC News NOW NBC News NOW NBC News NOW MORE AI NEWS Taylor Swift criticized a viral AI image of her 'endorsing' Trump. The AI image’s creator spoke to NBC News. Google to relaunch controversial tool for creating AI images after inaccuracies ------------------------------------------------------------------------------- Trump shares AI images of Taylor Swift and her fans supporting him ------------------------------------------------------------------ X’s AI image generator debuts with few guardrails on political content ----------------------------------------------------------------------"}, {'url'

In [ ]:
type(result.raw)

In [ ]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools

from langchain_community.tools import DuckDuckGoSearchResults

# Instantiate tools
search_tool = DuckDuckGoSearchResults(
    max_results=3,
)

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI in Healthcare industry and AI in Finance industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool],
    llm=llm,
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI in Healthcare industry and AI in Finance industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments with a unique perspective on their significance.',
    agent=researcher
)


# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher],
    tasks=[research],
    llm=llm,
    verbose=True,
    # planning=True,  # Enable planning feature
)

# Execute tasks
crew.kickoff()

### Crew with multiple agents and multiple tasks

#### Input  placeholder

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchResults

news_research_agent = Agent(
  role='News Researcher',
  goal='Find and summarize the latest news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  llm=llm,
  verbose=True
)

stock_research_agent = Agent(
  role='Stock Researcher',
  goal='Find and summarize the latest information on stocks',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  llm=llm,
  verbose=True
)

search_tool = DuckDuckGoSearchResults()

# Adding placeholder for input to the task
task_1 = Task(
  description='Find and summarize the latest stock market news on {topic}',
  expected_output='A bullet list summary of the important news',
  agent=news_research_agent,
  tools=[search_tool]
)

task_3 = Task(
  description="""First, identify the ticker symbol of companies (if present) present in the retrieved news. 
  Then analyse the trend of those stocks and summarize the information""",
  expected_output='A bullet list summary of the top stocks mentioned in the news',
  agent=stock_research_agent,
  tools=[search_tool]
)

crew = Crew(
    agents=[stock_research_agent, news_research_agent],
    tasks=[task_1, task_3],
    verbose=True,
    iterations=3
)

# Adding inputs to the task
result = crew.kickoff(inputs={'topic': 'nifty stocks'})
print(result)

#### Passing context from one task to another

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchResults

news_research_agent = Agent(
  role='News Researcher',
  goal='Find and summarize the latest news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  llm=llm,
  verbose=True
)

stock_research_agent = Agent(
  role='Stock Researcher',
  goal='Find and summarize the latest information on stocks',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  llm=llm,
  verbose=True
)

search_tool = DuckDuckGoSearchResults()

# Adding placeholder for input to the task
task_1 = Task(
  description='Find and summarize the latest stock market news on {topic}',
  expected_output='A bullet list summary of the important news',
  agent=news_research_agent,
  tools=[search_tool]
)

task_3 = Task(
  description="""First, identify the ticker symbol of companies (if present) present in the retrieved news. 
  Then analyse the trend of those stocks and summarize the information""",
  expected_output='A bullet list summary of the top stocks mentioned in the news',
  agent=stock_research_agent,
  tools=[search_tool],
  # Adding context to the task
  context=[task_1]
)

crew = Crew(
    agents=[stock_research_agent, news_research_agent],
    tasks=[task_1, task_3],
    verbose=True,
    iterations=3
)

# Adding inputs to the task
result = crew.kickoff(inputs={'topic': 'top Indian IT stocks'})
print(result)

In [12]:
task_3_output = task_3.output

In [ ]:
dict(task_3_output)

#### Passing task output as formatted information

In [14]:
from pydantic import BaseModel
from typing import List

class Stock(BaseModel):
    ticker: str
    trend: str
    news: str
    sentiment: str

class Output(BaseModel):
    stocks: List[Stock]

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import ScrapeWebsiteTool

news_research_agent = Agent(
  role='News Researcher',
  goal='Find and summarize the latest news.',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing news and providing insights to the business.""",
  llm=llm,
  verbose=True
)

stock_research_agent = Agent(
  role='Stock Researcher',
  goal='Find and summarize the latest information on stocks',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing stocks data and providing insights to the business.""",
  llm=llm,
  verbose=True
)

new_search_tool = DuckDuckGoSearchResults(
    max_results=5,
)

stock_news_search_tool = DuckDuckGoSearchResults(
    max_results=1,
    search_query='site:moneycontrol.com'
)

scrape_tool = ScrapeWebsiteTool()

# Adding placeholder for input to the task
task_1 = Task(
  description='Find and summarize the latest news on {topic}',
  expected_output='A bullet list summary of the important news',
  agent=news_research_agent,
  tools=[new_search_tool, scrape_tool]
)

task_3 = Task(
  description="""First, identify the ticker symbol of companies (if present) present in the retrieved news. 
  Then analyse the trend of those stocks and summarize the information""",
  expected_output='A bullet list summary of all the stocks mentioned in the news',
  agent=stock_research_agent,
  tools=[stock_news_search_tool],
  # Task output Formatting
  # output_pydantic=Output,
  output_json=Output,
  # Adding context to the task
  context=[task_1]
)

crew = Crew(
    agents=[stock_research_agent, news_research_agent],
    tasks=[task_1, task_3],
    verbose=True,
    iterations=3
)

# Adding inputs to the task
result = crew.kickoff(inputs={'topic': 'SENSEX and NIFTY'})
print(result)

In [ ]:
task_3_output = task_3.output
dict(task_3_output)

In [ ]:
if task_3_output.pydantic:
    print(task_3_output.pydantic)
elif task_3_output.json:
    print(task_3_output.json)

In [ ]:
task_1_output = task_1.output
dict(task_1_output)

#### Adding Callbacks to the Task for better tracking

In [19]:
from crewai.tasks.task_output import TaskOutput
def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(f"""
        Task completed!
        Agent: {output.agent}
        Task: {output.description}
        Output: {output.raw}
    """)

In [20]:
from pydantic import BaseModel
from typing import List

class Stock(BaseModel):
    ticker: str
    trend: str
    news: str
    sentiment: str

class Output(BaseModel):
    stocks: List[Stock]

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import ScrapeWebsiteTool

news_research_agent = Agent(
  role='News Researcher',
  goal='Provide up-to-date market analysis.',
  backstory='An expert analyst with a keen eye for market trends.',
  llm=llm,
  # verbose=True
)

stock_research_agent = Agent(
  role='Stock Researcher',
  goal='Provide up-to-date market analysis of companies/stocks mentioned in the news.',
  backstory='An expert analyst with a keen eye for market trends.',
  llm=llm,
  # verbose=True
)

new_search_tool = DuckDuckGoSearchResults(
    max_results=3,
)

stock_news_search_tool = DuckDuckGoSearchResults(
    max_results=1,
)

scrape_tool = ScrapeWebsiteTool()

task_1 = Task(
  description='Search, read and summarize the latest stock market news on {topic}',
  expected_output='A bullet list summary of the important news',
  agent=news_research_agent,
  tools=[new_search_tool, scrape_tool],
  # Callback function
  callback=callback_function
)

task_3 = Task(
  description="""First, identify the ticker symbol of companies (if present) present in the retrieved news. 
  Then analyse the news to undertand the trend of those stocks.""",
  expected_output='A bullet list summary of all the stocks mentioned in the news',
  agent=stock_research_agent,
  tools=[stock_news_search_tool],
  # output_pydantic=Output,
  output_json=Output,
  context=[task_1],
  # Callback function
  callback=callback_function
)

crew = Crew(
    agents=[stock_research_agent, news_research_agent],
    tasks=[task_1, task_3],
    # verbose=True,
    iterations=3
)

# Adding inputs to the task
result = crew.kickoff(inputs={'topic': 'NIFTY50 companies'})
print(result)

In [ ]:
task_3.output.json_dict

### Exporing the processes

#### Sequential Process

In [23]:
from crewai.tasks.task_output import TaskOutput
def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(f"""
        Task completed!
        Agent: {output.agent}
        Task: {output.description}
        Output: {output.raw}
    """)

In [24]:
from pydantic import BaseModel, Field
from typing import List

class Companies(BaseModel):
    company_name: str
    ticker: str 
    news: List[str]

class Company_list(BaseModel):
    companies: List[Companies]

class Stock_trends(BaseModel):
    ticker: str
    trend: str = Field(description="The trend of the stock", example="Bullish")
    news: List[str]
    sentiment: str

class Output(BaseModel):
    stocks: List[Stock_trends]

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import ScrapeWebsiteTool, WebsiteSearchTool
from crewai.process import Process

news_search_tool = DuckDuckGoSearchResults(
    max_results=5,
)

stock_news_search_tool = DuckDuckGoSearchResults(
    max_results=1,
)

scrape_tool = ScrapeWebsiteTool()
web_rag_tool = WebsiteSearchTool()

news_research_agent = Agent(
  role='News Researcher',
  goal='Read and provide up-to-date news analysis.',
  backstory='An expert researcher with a keen eye for details.',
  tools=[news_search_tool, scrape_tool],
  llm=llm,
  verbose=True
)

stock_research_agent = Agent(
  role='Stock Researcher',
  goal='Provide up-to-date market analysis of companies/stocks.',
  backstory='An expert analyst with a keen eye for market trends.',
  llm=llm,
)

task_1 = Task(
  description="""Given an input company or sector or index: {portfolio} and date: {date},
  Generate 1 relevant search query, search moneycontrol.com, scrape/read and summarise the news.""",
  expected_output="A list of summarised news reports.",
  agent=news_research_agent,
  callback=callback_function
)

task_2 = Task(
  description="""identify the companies present in the retrieved news. 
  Using search tool, find the ticker symbol of the companies and top news of the day.""",
  expected_output='A list of companies, their ticker symbol and their top news of the day.',
  tools=[stock_news_search_tool],
  agent=news_research_agent,
  output_json=Company_list,
  callback=callback_function
)

task_3 = Task(
  description="""Analyse the news to undertand the trend and sentiment of all the companies.""",
  expected_output='Details of all the stocks mentioned in the news',
  agent=stock_research_agent,
  output_pydantic=Output,
  callback=callback_function
)

crew = Crew(
    agents=[
        news_research_agent,
        # stock_research_agent, 
      ],
    
    tasks=[
        task_1, 
        # task_2, 
        # task_3
      ],
    verbose=True,
    # iterations=3,
    # Makin the process sequential
    # ask execution follows the predefined order in the task list, with the output of one task serving as context for the next.
    process=Process.sequential
)

# Adding inputs to the task
portfolio = 'NIFTY50'
date =datetime.date.today().strftime("%B %d, %Y")
result = crew.kickoff(inputs={'portfolio': portfolio, 'date': date})
print(result)

In [ ]:
task_2.output.json_dict

In [ ]:
task_3.output.pydantic

#### Hierarchical Process

In [ ]:
from crewai.tasks.task_output import TaskOutput
def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(f"""
        Task completed!
        Agent: {output.agent}
        Task: {output.description}
        Output: {output.raw}
    """)

In [ ]:
from pydantic import BaseModel
from typing import List

class News(BaseModel):
    company_name: str
    news: str

class Stock_news(BaseModel):
    news: List[News]

class Companies(BaseModel):
    company_name: str
    ticker: str

class Company_list(BaseModel):
    companies: List[Companies]

class Stock_trends(BaseModel):
    ticker: str
    trend: str
    news: List[str]
    sentiment: str

class Output(BaseModel):
    stocks: List[Stock_trends]

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import ScrapeWebsiteTool
from crewai.process import Process

news_search_tool = DuckDuckGoSearchResults(
    max_results=3,
)

stock_news_search_tool = DuckDuckGoSearchResults(
    max_results=1,
)

scrape_tool = ScrapeWebsiteTool()

news_research_agent = Agent(
  role='News Researcher',
  goal= "First, Generate the most relevant search query. Then efficiently retrieve accurate and relevant search results for each query.",
  backstory= """I am a dedicated Researcher with a deep understanding of information retrieval and analysis. 
                My expertise lies in transforming a user's query into the most relevant search questions and retrieving results.""",
  tools=[news_search_tool, news_search_tool],
  llm=llm,
  verbose=True
)

news_aggregator_agent = Agent(
  role='News Aggregator',
  goal= "Aggregate and summarize the news from the search results.",
  backstory= """I am a dedicated News Aggregator with a deep understanding of stock market and business. 
                My expertise lies in transforming news into ationable insights.""",   
  llm=llm,
)

stock_research_agent = Agent(
  role='Stock Researcher',
  goal='Provide up-to-date market analysis of companies/stocks.',
  backstory='An expert analyst with a keen eye for market trends.',
  llm=llm,
)

task_1 = Task(
  description="""Given a protfolio of companies: {portfolio}, first identify all the company names.
  Then generate search query for each company in the portfolio, scrape the news from the search results.
  Finally compile all the search results and return the company name and the news for each company.""",
  expected_output="All the companies mentoined in the portfolio and summary of retrieved news reports.",
  agent=news_research_agent,
  output_json=Stock_news,
  # callback=callback_function
)

task_2 = Task(
  description="""identify the companies present in the retrieved news. 
  Using search tool, find the ticker symbol of the companies.""",
  expected_output='A list of companies and their ticker symbol.',
  tools=[stock_news_search_tool],
  agent=news_research_agent,
  output_json=Company_list,
  callback=callback_function
)

task_3 = Task(
  description="""Analyse the news to undertand the trend and sentiment of all the companies.""",
  expected_output='Details of all the stocks mentioned in the news',
  agent=stock_research_agent,
  output_pydantic=Output,
  context=[task_1, task_2],
  callback=callback_function
)

crew = Crew(
    agents=[
        news_research_agent,
        news_aggregator_agent,
        # stock_research_agent, 
      ],
    
    tasks=[
        task_1, 
        # task_2, 
        # task_3
      ],
    verbose=True,
    # iterations=3,
    # Makin the process sequential
    # ask execution follows the predefined order in the task list, with the output of one task serving as context for the next.
    process=Process.hierarchical
)

# Adding inputs to the task
portfolio = ['IT Stocks']
result = crew.kickoff(inputs={'portfolio': str(', '.join(portfolio))})
print(result)

In [ ]:
task_1.output.json_dict

In [ ]:
task_2.output.json_dict

In [ ]:
task_3.output.pydantic

### Crew